In [10]:
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    value: str

def node_a(state: State):
    print("NODE A")
    return Command(
        goto="node_b",
        update={
            "value": state["value"] + "a",
        }
    )

def node_b(state: State):
    print("NODE B")
    human_response = interrupt("Do you want to go to C or D?")
    print(f"human response: {human_response}")

    if human_response.lower() == "c":
        return Command(
            goto="node_c",
            update={state["value"] + "c"}
        )
    elif human_response.lower() == "d":
        return Command(
            goto="node_d",
            update={state["value"] + "d"}
        )
    else:
        return Command(
            goto="node_b"
        )

def node_c(state: State):
    print("NODE C")
    return Command(
        goto=END,
        update={state["value"] + "c"}
    )

def node_d(state: State):
    print("NODE D")
    return Command(
        goto=END,
        update={state["value"] + "d"}
    )

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)
graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "1"}}

initial_state = {"value": ""}

first_result = app.invoke(initial_state, config, stream_mode="updates")
first_result


NODE A
NODE B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D?', resumable=True, ns=['node_b:d9cfeeff-8e18-ec07-42a4-79fa75008be1']),)}]

In [11]:
app.get_state(config).next

('node_b',)

In [13]:
second_result = app.invoke(Command(resume="c"), config=config, stream_mode="updates")
second_result

NODE B
human response: c
NODE C


[{'node_b': None}, {'node_c': None}]